In [ ]:
# (실습 1) 네이버 쇼핑에서 ‘노트북’ 상품 정보 가져오기
import requests
from bs4 import BeautifulSoup
import re

url = 'https://search.shopping.naver.com/search/all?query=%EB%85%B8%ED%8A%B8%EB%B6%81&frm=NVSHATC&prevQuery=%EB%85%B8%ED%8A%B8%EB%B6%81'
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

res = requests.get(url)
print(res.status_code)
res.raise_for_status()
soup = BeautifulSoup(res.text, 'lxml')

# print(res.text)
with open('shopping.html', 'w', encoding='utf8') as f:
    # f.write(res.text)
    f.write(soup.prettify())    # html 문서를 예쁘게 출력

In [ ]:
# (실습 2) ‘노트북’ 상품명, 가격, 평점, 리뷰수 가져오기
import requests
from bs4 import BeautifulSoup
import re

url = 'https://search.shopping.naver.com/search/all?query=%EB%85%B8%ED%8A%B8%EB%B6%81&frm=NVSHATC&prevQuery=%EB%85%B8%ED%8A%B8%EB%B6%81'

res = requests.get(url)
print(res.status_code)
res.raise_for_status()
soup = BeautifulSoup(res.text, 'lxml')

items = soup.find_all('li', attrs={'class':'basicList_item__2XT81'})
print('상품 개수:', len(items))

for i in items:
    name = i.find('a', attrs={'class':'basicList_link__1MaTN'}).get_text()                                  # 제품명
    price = i.find('span', attrs={'class':'price_num__2WUXn'}).get_text()                                   # 가격
    rating = i.find('span', attrs={'class':'basicList_star__3NkBn'}).get_text().replace('별점', '').strip() # 평점
    review_count = i.find('em', attrs={'class':'basicList_num__1yXM9'}).get_text()                          # 리뷰수
    print(name, price, rating, review_count)


In [ ]:
# (실습 3) 동적 페이지 => selenium으로 페이지 맨 아랫부분까지 스크롤 내리기
from selenium import webdriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://search.shopping.naver.com/search/all?query=%EB%85%B8%ED%8A%B8%EB%B6%81&frm=NVSHATC&prevQuery=%EB%85%B8%ED%8A%B8%EB%B6%81") 
import time

SCROLL_PAUSE_TIME = 1

# Get scroll height => 현재 문서의 높이 저장
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom => 현재 문서의 높이만큼 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

print('스크를 내리기 성공')

driver.close()



In [ ]:
# (실습 4) 현재 페이지의 상품 정보 출력하기
from selenium import webdriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://search.shopping.naver.com/search/all?query=%EB%85%B8%ED%8A%B8%EB%B6%81&frm=NVSHATC&prevQuery=%EB%85%B8%ED%8A%B8%EB%B6%81") 
import time
from bs4 import BeautifulSoup
import re

#region 1. 화면 제일 아래까지 스크롤 내리기
SCROLL_PAUSE_TIME = 1

# Get scroll height => 현재 문서의 높이 저장
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom => 현재 문서의 높이만큼 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

#endregion


soup = BeautifulSoup(driver.page_source, 'lxml')

items = soup.find_all('li', attrs={'class':'basicList_item__2XT81'})
print('상품 개수:', len(items))

for i in items:
    name = i.find('a', attrs={'class':'basicList_link__1MaTN'}).get_text()          # 제품명
    price = i.find('span', attrs={'class':'price_num__2WUXn'}).get_text()           # 가격
    rating = i.find('span', attrs={'class':'basicList_star__3NkBn'})                # 평점                          
    if rating:
        rating = rating.get_text().replace('별점', '').strip()
    else:
        print('평점 없는 상품 제외됨.')
        continue
    review_count = i.find('em', attrs={'class':'basicList_num__1yXM9'})              # 리뷰수
    if review_count:
        review_count = review_count.get_text().replace(',','').strip()
    else:
        print('리뷰수 없는 상품 제외')
        continue
    link = i.find('a', attrs={'class':'basicList_link__1MaTN'})['href']                                     # 링크
    print(name, price, rating, review_count, link)

driver.close()

In [ ]:
# (실습 5) 광고 제품 제외하기
from selenium import webdriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://search.shopping.naver.com/search/all?query=%EB%85%B8%ED%8A%B8%EB%B6%81&frm=NVSHATC&prevQuery=%EB%85%B8%ED%8A%B8%EB%B6%81") 
import time
from bs4 import BeautifulSoup
import re

#region 1. 화면 제일 아래까지 스크롤 내리기
SCROLL_PAUSE_TIME = 1

# Get scroll height => 현재 문서의 높이 저장
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom => 현재 문서의 높이만큼 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

#endregion


soup = BeautifulSoup(driver.page_source, 'lxml')

items = soup.find_all('li', attrs={'class':'basicList_item__2XT81'})
print('상품 개수:', len(items))

for i in items:
    # 광고 상품 제외하기
    is_ad_item = i.find('button', attrs={'class':'ad_ad_stk__12U34'})
    if is_ad_item:
        print('광고 상품 제외됨.')
        continue
    
    name = i.find('a', attrs={'class':'basicList_link__1MaTN'}).get_text()          # 제품명
    price = i.find('span', attrs={'class':'price_num__2WUXn'}).get_text()           # 가격
    rating = i.find('span', attrs={'class':'basicList_star__3NkBn'})                # 평점                          
    if rating:
        rating = rating.get_text().replace('별점', '').strip()
    else:
        print('평점 없는 상품 제외됨.')
        continue
    review_count = i.find('em', attrs={'class':'basicList_num__1yXM9'})              # 리뷰수
    if review_count:
        review_count = review_count.get_text().replace(',','').strip()
    else:
        print('리뷰수 없는 상품 제외')
        continue
    link = i.find('a', attrs={'class':'basicList_link__1MaTN'})['href']                                     # 링크
    print(name, price, rating, review_count, link)

driver.close()


In [ ]:
# (실습 5) 광고 제품, Apple 제품 제외하기
from selenium import webdriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://search.shopping.naver.com/search/all?query=%EB%85%B8%ED%8A%B8%EB%B6%81&frm=NVSHATC&prevQuery=%EB%85%B8%ED%8A%B8%EB%B6%81") 
import time
from bs4 import BeautifulSoup
import re

#region 1. 화면 제일 아래까지 스크롤 내리기
SCROLL_PAUSE_TIME = 1

# Get scroll height => 현재 문서의 높이 저장
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom => 현재 문서의 높이만큼 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

#endregion


soup = BeautifulSoup(driver.page_source, 'lxml')

items = soup.find_all('li', attrs={'class':'basicList_item__2XT81'})
print('상품 개수:', len(items))

for i in items:
    # 광고 상품 제외하기
    is_ad_item = i.find('button', attrs={'class':'ad_ad_stk__12U34'})
    if is_ad_item:
        print('광고 상품 제외됨.')
        continue
    
    name = i.find('a', attrs={'class':'basicList_link__1MaTN'}).get_text()          # 제품명
    # Apple 제품 제외하기
    if "Apple" in name:
        print('Apple 상품 제외됨.')
        continue

    price = i.find('span', attrs={'class':'price_num__2WUXn'}).get_text()           # 가격
    rating = i.find('span', attrs={'class':'basicList_star__3NkBn'})                # 평점                          
    if rating:
        rating = rating.get_text().replace('별점', '').strip()
    else:
        print('평점 없는 상품 제외됨.')
        continue
    review_count = i.find('em', attrs={'class':'basicList_num__1yXM9'})              # 리뷰수
    if review_count:
        review_count = review_count.get_text().replace(',','').strip()
    else:
        print('리뷰수 없는 상품 제외')
        continue
    link = i.find('a', attrs={'class':'basicList_link__1MaTN'})['href']                                     # 링크
    print(name, price, rating, review_count, link)

driver.close()

In [ ]:
# 최근 10개 페이지에서 모든 상품 정보 가져오기
# Apple 제품 제외하기
# 평점 없는 제품 제외하기
# 네이버 쇼핑에서 노트북 정보 가져오기
import requestsㅇㅇ
from bs4 import BeautifulSoup
import re

for i in range(1, 11):
    #아래의 모든 코드 들여쓰기, page 번호 변경 처리
    pass

url = 'https://search.shopping.naver.com/search/all?query=%EB%85%B8%ED%8A%B8%EB%B6%81&frm=NVSHATC&prevQuery=%EB%85%B8%ED%8A%B8%EB%B6%81'
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

res = requests.get(url)
print(res.status_code)
res.raise_for_status()
soup = BeautifulSoup(res.text, 'lxml')

items = soup.find_all('li', attrs={'class':'basicList_item__2XT81'})

for i in items:
    
    # 광고 상품 제외하기
    is_ad_item = i.find('button', attrs={'class':'ad_ad_stk__12U34'})
    if is_ad_item:
        print('광고 상품 제외됨.')
        continue
    
    name = i.find('a', attrs={'class':'basicList_link__1MaTN'}).get_text()          # 제품명
    # Apple 제품 제외하기
    if "Apple" in name:
        print('Apple 상품 제외됨.')
        continue
    
    price = i.find('span', attrs={'class':'price_num__2WUXn'}).get_text()           # 가격
    rating = i.find('span', attrs={'class':'basicList_star__3NkBn'})                # 평점                          
    if rating:
        rating = rating.get_text().replace('별점', '').strip()
    else:
        print('평점 없는 상품 제외됨.')
        continue
    
    review_count = i.find('em', attrs={'class':'basicList_num__1yXM9'})              # 리뷰수
    if review_count:
        review_count = review_count.get_text().replace(',','').strip()
    else:
        print('리뷰수 없는 상품 제외')
        continue
    
    link = i.find('a', attrs={'class':'basicList_link__1MaTN'})['href']                                     # 링크
    
    # 평점 4.5 이상, 리뷰수 5000개 이상 제품만 남기기
    if float(rating)>=4.5 and int(review_count)>=5000:
        print(name, price, rating, review_count, link)


In [ ]:
# 동적 페이지 selenium으로 페이지 맨 아랫부분까지 스크롤 내리기
from selenium import webdriver
driver = webdriver.Chrome()
driver.maximize_window()
import time
from bs4 import BeautifulSoup

item_list=[]
for page in range(1, 6):
    url = 'https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%EB%85%B8%ED%8A%B8%EB%B6%81&pagingIndex={}&pagingSize=40&productSet=total&query=%EB%85%B8%ED%8A%B8%EB%B6%81&sort=rel&timestamp=&viewType=list'.format(page)
    driver.get(url) 

    #region 1. 화면의 제일 아랫 부분까지 스크롤 내리기
    SCROLL_PAUSE_TIME = 1

    # Get scroll height => 현재 문서의 높이 저장
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom => 현재 문서의 높이만큼 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    print('스크를 내리기 성공')
    #endregion

    # 2. 스크롤된 페이지 파싱
    soup = BeautifulSoup(driver.page_source, 'lxml')

    #region 3. 상품 정보 추출하기
    items = soup.find_all('li', attrs={'class':'basicList_item__2XT81'})    # 상품 정보
    print(len(items))

    for i in items:
        # 광고 상품 제외하기
        is_ad_item = i.find('button', attrs={'class':'ad_ad_stk__12U34'})
        if is_ad_item:
            print('광고 상품 제외됨.')
            continue
        
        name = i.find('a', attrs={'class':'basicList_link__1MaTN'}).get_text()          # 제품명
        # Apple 제품 제외하기
        if "Apple" in name:
            print('Apple 상품 제외됨.')
            continue
        
        price = i.find('span', attrs={'class':'price_num__2WUXn'}).get_text()           # 가격
        
        rating = i.find('span', attrs={'class':'basicList_star__3NkBn'})                # 평점                          
        if rating:
            rating = rating.get_text().replace('별점', '').strip()
        else:
            print('평점 없는 상품 제외됨.')
            continue
        
        review_count = i.find('em', attrs={'class':'basicList_num__1yXM9'})              # 리뷰수
        if review_count:
            review_count = review_count.get_text().replace(',','').strip()
        else:
            print('리뷰수 없는 상품 제외')
            continue
        
        link = i.find('a', attrs={'class':'basicList_link__1MaTN'})['href']                                     # 링크
        
        item_list.append(f'{name}, {price}, {rating}, {review_count}')
        #endregion
        
    time.sleep(1)
    
print(len(item_list))


In [ ]:
# (실습 7) csv 파일로 저장하기
from selenium import webdriver
driver = webdriver.Chrome()
driver.maximize_window()
import time
from bs4 import BeautifulSoup
import csv

# 1. csv 파일 생성
file_name = 'notebook.csv'
f = open(file_name, 'w', encoding='utf8', newline='')
writer = csv.writer(f)

for page in range(1, 6):
    url = 'https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%EB%85%B8%ED%8A%B8%EB%B6%81&pagingIndex={}&pagingSize=40&productSet=total&query=%EB%85%B8%ED%8A%B8%EB%B6%81&sort=rel&timestamp=&viewType=list'.format(page)
    driver.get(url) 


    #region 2. 화면의 제일 아랫 부분까지 스크롤 내리기
    SCROLL_PAUSE_TIME = 1

    # Get scroll height => 현재 문서의 높이 저장
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom => 현재 문서의 높이만큼 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    print('스크를 내리기 성공')
    #endregion

    # 3. 스크롤된 페이지 파싱
    soup = BeautifulSoup(driver.page_source, 'lxml')

    #region 4. 상품 정보 추출하기
    items = soup.find_all('li', attrs={'class':'basicList_item__2XT81'})    # 상품 정보
    print(len(items))

    for i in items:
        # 광고 상품 제외하기
        is_ad_item = i.find('button', attrs={'class':'ad_ad_stk__12U34'})
        if is_ad_item:
            print('광고 상품 제외됨.')
            continue
        
        name = i.find('a', attrs={'class':'basicList_link__1MaTN'}).get_text()          # 제품명
        # Apple 제품 제외하기
        if "Apple" in name:
            print('Apple 상품 제외됨.')
            continue
        
        price = i.find('span', attrs={'class':'price_num__2WUXn'}).get_text()           # 가격
        
        rating = i.find('span', attrs={'class':'basicList_star__3NkBn'})                # 평점                          
        if rating:
            rating = rating.get_text().replace('별점', '').strip()
        else:
            print('평점 없는 상품 제외됨.')
            continue
        
        review_count = i.find('em', attrs={'class':'basicList_num__1yXM9'})              # 리뷰수
        if review_count:
            review_count = review_count.get_text().replace(',','').strip()
        else:
            print('리뷰수 없는 상품 제외')
            continue
        
        link = i.find('a', attrs={'class':'basicList_link__1MaTN'})['href']                                     # 링크

        writer.writerow([name, price, rating, review_count, link])   
    time.sleep(1)  
    #endregion

f.close()
driver.close()

In [ ]:
# (실습 8) 제품 검색 조건 추가하기
from selenium import webdriver
driver = webdriver.Chrome()
driver.maximize_window()
import time
from bs4 import BeautifulSoup
import csv

# 1. csv 파일 생성
file_name = 'notebook.csv'
f = open(file_name, 'w', encoding='utf8', newline='')
writer = csv.writer(f)

for page in range(1, 6):
    url = 'https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%EB%85%B8%ED%8A%B8%EB%B6%81&pagingIndex={}&pagingSize=40&productSet=total&query=%EB%85%B8%ED%8A%B8%EB%B6%81&sort=rel&timestamp=&viewType=list'.format(page)
    driver.get(url) 


    #region 2. 화면의 제일 아랫 부분까지 스크롤 내리기
    SCROLL_PAUSE_TIME = 1

    # Get scroll height => 현재 문서의 높이 저장
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom => 현재 문서의 높이만큼 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    print('스크를 내리기 성공')
    #endregion

    # 3. 스크롤된 페이지 파싱
    soup = BeautifulSoup(driver.page_source, 'lxml')

    #region 4. 상품 정보 추출하기
    items = soup.find_all('li', attrs={'class':'basicList_item__2XT81'})    # 상품 정보
    print(len(items))

    for i in items:
        # 광고 상품 제외하기
        is_ad_item = i.find('button', attrs={'class':'ad_ad_stk__12U34'})
        if is_ad_item:
            print('광고 상품 제외됨.')
            continue
        
        name = i.find('a', attrs={'class':'basicList_link__1MaTN'}).get_text()          # 제품명
        # Apple 제품 제외하기
        if "Apple" in name:
            print('Apple 상품 제외됨.')
            continue
        
        price = i.find('span', attrs={'class':'price_num__2WUXn'}).get_text()           # 가격
        
        rating = i.find('span', attrs={'class':'basicList_star__3NkBn'})                # 평점                          
        if rating:
            rating = rating.get_text().replace('별점', '').strip()
        else:
            print('평점 없는 상품 제외됨.')
            continue
        
        review_count = i.find('em', attrs={'class':'basicList_num__1yXM9'})              # 리뷰수
        if review_count:
            review_count = review_count.get_text().replace(',','').strip()
        else:
            print('리뷰수 없는 상품 제외')
            continue
        
        link = i.find('a', attrs={'class':'basicList_link__1MaTN'})['href']                                     # 링크

        # 평점 4.5 이상, 리뷰수 500개 이상 제품만 저장하기
        if float(rating)>=4.5 and int(review_count)>=500:
            writer.writerow([name, price, rating, review_count, link])  
         
    time.sleep(1)  
    #endregion

f.close()
driver.close()